# Template de submissão

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intrig-unicamp/hackathon5G/blob/main/challenges/submission-template.ipynb)

Esse Notebook importa todas as dependências necessárias para trabalhar com os conjuntos de dados fornecidos pela comissão organizadora da Hackathon SMARTNESS. As equipes poderão utilizá-lo como base para as submissões.

## Instalação das dependências

Instalar as dependências no ambiente adequado (pip ou Anaconda).

In [1]:
try:
    import google.colab as _
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install pyshark pandas numpy seaborn scikit-learn plotly -q
else:
    !conda install pyshark pandas numpy seaborn scikit-learn plotly -qy --freeze-installed

In [2]:
%%bash

if type tshark >/dev/null 2>&1; then
    echo "tshask already installed"

elif type nix-env >/dev/null 2>&1; then
    nix-env -iA nixpkgs.tshark

elif type apt-get >/dev/null 2>&1; then
    apt-get -q install tshark

elif type yum >/dev/null 2>&1; then
    yum install tshark -q -y

elif type apk >/dev/null 2>&1; then
    apk add -q tshark

elif type pacman >/dev/null 2>&1; then
    pacman -S tshark

fi

tshask already installed


## Obtendo arquivos de dados

Baixamos os arquivos caso o Notebook esteja sendo executado no ambiente do Google Colab. A variável de prefixo do caminho dos arquivos é definida de acordo.

In [3]:
if IN_COLAB:
    !git clone --depth=1 https://github.com/intrig-unicamp/hackathon5G.git hackathon5G

DATA_PATH_PREFIX = './hackathon5G/datasets' if IN_COLAB else '../datasets'

## Importando dependências globais

In [4]:
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances

import pyshark

# https://github.com/KimiNewt/pyshark/issues/360#issuecomment-700425352
import nest_asyncio
nest_asyncio.apply()

import os
from datetime import datetime
from math import radians
import json
from urllib.parse import urlparse, parse_qsl

pd.set_option('display.max_columns', None)

## Carregando os dados

### Mosaico

In [5]:
MOSAICO_PATH = f'{DATA_PATH_PREFIX}/mosaico'

regions = [
    'são-paulo',
    'campinas',
    'rio-de-janeiro',
    'brasília',
]

ERBs = pd.concat([ pd.read_csv(f'{MOSAICO_PATH}/mosaico-erbs-{region}.zip', encoding='iso-8859-1', low_memory=False) for region in regions ], ignore_index=True).copy()
ERBs.replace({ 'Tecnologia': { 'NR ': 'NR', 'WDCMA': 'WCDMA' } }, inplace=True)
ERBs['Tecnologia_gen'] = ERBs.Tecnologia.map({ 'GSM': '2G', 'WCDMA': '3G', 'LTE': '4G', 'NR': '5G' })
ERBs.Azimute = ERBs.Azimute.str.replace(',', '.').replace('V', np.nan).astype(np.float64)
ERBs['Azimute_rad'] = ERBs.Azimute.apply(lambda val: val * np.pi/180)
ERBs

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,NumEstacao,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,Tecnologia,tipoTecnologia,meioAcesso,FreqTxMHz,FreqRxMHz,Azimute,CodTipoClasseEstacao,ClassInfraFisica,CompartilhamentoInfraFisica,CodTipoAntena,CodEquipamentoAntena,GanhoAntena,FrenteCostaAntena,AnguloMeiaPotenciaAntena,AnguloElevacao,Polarizacao,AlturaAntena,CodEquipamentoTransmissor,PotenciaTransmissorWatts,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado,Tecnologia_gen,Azimute_rad
0,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,17012008.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,5M00G9W,WCDMA,NaN,NaN,2160.0,1970.0,230.0,FB,NaN,NaN,760,002000703518,14.9,25,70,6,X,41,018930701882,39.4,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6a,2023-04-30,NaN,NaN,3G,4.014257
1,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,360.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6b,2023-04-30,NaN,NaN,2G,6.283185
2,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1862.5,1767.5,130.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6c,2023-04-30,NaN,NaN,2G,2.268928
3,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,130.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6d,2023-04-30,NaN,NaN,2G,2.268928
4,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146447,10,35312011.0,122700,Rua Manoel Duque de Carvalho,NaN,SP,3550308,200KG7W,GSM,NaN,NaN,1867.5,1772.5,240.0,FB,NaN,NaN,760,038801200762,17.8,25,65,5,X,39,006030301882,15.8,-23.71889,-46.68889,G,2021-12-24,1999-12-10,NaN,4d5c019f58e6e,2023-04-30,NaN,NaN,2G,4.188790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218865,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,35312011.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,1855.0,1760.0,NaN,FB,NaN,NaN,760,NaN,13,20.0,63,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc02039,2023-04-30,NaN,NaN,4G,NaN
1218866,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,17012008.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,2160.0,1970.0,NaN,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203a,2023-04-30,NaN,NaN,4G,NaN
1218867,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,17012008.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,10M0G7W,LTE,NaN,NaN,2160.0,1970.0,NaN,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203b,2023-04-30,NaN,NaN,4G,NaN
1218868,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,50852013.0,1014783507,SHIS QI. 12 CONJUNTO 8/9,NaN,DF,5300108,5M00G7W,LTE,NaN,NaN,2167.5,1977.5,NaN,FB,NaN,NaN,760,NaN,13.3,20.0,62,0.0,X,4.0,083071803257,40,-15.83168,-47.86179,G,2023-03-21,2023-03-21,NaN,5ae855f89dc0203c,2023-04-30,NaN,NaN,4G,NaN


### G-NetTrack

In [6]:
GNETTRACKPRO_PATH = f'{DATA_PATH_PREFIX}/g-nettrack-pro'

datasets = [ file for file in os.listdir(GNETTRACKPRO_PATH) if file.endswith('.txt') ]

gnetDf = pd.concat([ pd.read_csv(f'{GNETTRACKPRO_PATH}/{dataset}', sep='\t', low_memory=False, on_bad_lines='warn') for dataset in datasets ]).copy()
gnetDf.drop(gnetDf.loc[gnetDf.Timestamp == 'Timestamp'].index, inplace=True)
gnetDf.Timestamp = pd.to_datetime(gnetDf.Timestamp, format='%Y.%m.%d_%H.%M.%S')
gnetDf['Timestamp_seconds'] = gnetDf.Timestamp.astype(int) // 10**9
gnetDf.sort_values('Timestamp', inplace=True, ignore_index=True)
gnetDf.replace({
    'Altitude': { '--': np.nan },
    'CHARGING': { 'true': True, 'false': False },
    **{ k: { '--': np.nan } for k in ['Longitude', 'Latitude', 'NetworkTech', 'Accuracy'] },
    **{ k: { '-': np.nan } for k in ['CQI', 'SNR', 'Qual', 'LTERSSI'] },
}, inplace=True)
gnetDf = gnetDf.astype({
    **{ k: 'string' for k in ['NetworkTech', 'NetworkMode', 'Location', 'State', 'EVENT', 'CGI', 'Node', 'CellID', 'LAC', 'Operatorname', 'Operator', 'IP'] },
    **{ k: np.float64 for k in ['Longitude', 'Latitude', 'Speed', 'Accuracy', 'SNR', 'Level', 'Qual', 'CQI', 'LTERSSI', 'ARFCN', 'DL_bitrate', 'UL_bitrate', 'PSC', 'Altitude', 'Height'] },
    **{ k: np.int32 for k in ['BATTERY'] },
})
gnetDf.replace({
    'Altitude': { 0: np.nan },
    'Height': { 0: np.nan, -10000: np.nan },
    'Speed': { -99: np.nan },
}, inplace=True)
gnetDf

Skipping line 1291: expected 260 fields, saw 390
Skipping line 1292: expected 260 fields, saw 390
Skipping line 1293: expected 260 fields, saw 390
Skipping line 1294: expected 260 fields, saw 390
Skipping line 1295: expected 260 fields, saw 390
Skipping line 1296: expected 260 fields, saw 390
Skipping line 1297: expected 260 fields, saw 390
Skipping line 1298: expected 260 fields, saw 390
Skipping line 1299: expected 260 fields, saw 390
Skipping line 1300: expected 260 fields, saw 390
Skipping line 1301: expected 260 fields, saw 390
Skipping line 1302: expected 260 fields, saw 390
Skipping line 1303: expected 260 fields, saw 390
Skipping line 1304: expected 260 fields, saw 390
Skipping line 1305: expected 260 fields, saw 390
Skipping line 1306: expected 260 fields, saw 390
Skipping line 1307: expected 260 fields, saw 390
Skipping line 1308: expected 260 fields, saw 390
Skipping line 1309: expected 260 fields, saw 390
Skipping line 1310: expected 260 fields, saw 390
Skipping line 1311: 

,Timestamp,Longitude,Latitude,Speed,Operatorname,Operator,CGI,Cellname,Node,CellID,LAC,NetworkTech,NetworkMode,Level,Qual,SNR,CQI,LTERSSI,ARFCN,DL_bitrate,UL_bitrate,PSC,Altitude,Height,Accuracy,Location,State,PINGAVG,PINGMIN,PINGMAX,PINGSTDEV,PINGLOSS,TESTDOWNLINK,TESTUPLINK,TESTDOWNLINKMAX,TESTUPLINKMAX,Test_Status,DataConnection_Type,DataConnection_Info,Layer,Filemark,IMEI,Distance,Bearing,IP,TA,EVENT,EVENTDETAILS,IMSI,MSISDN,SERVINGTIME,CELLHEX,NODEHEX,LACHEX,RAWCELLID,VERSION_NAME,VERSION_CODE,DEVICE,BAND,BANDWIDTH,INFO,SecondCell_NODE,SecondCell_CELLID,SecondCell_RSRP,SecondCell_RSRQ,SecondCell_SNR,CSI_RSRP,CSI_RSRQ,CSI_SNR,NetworkTypeNum,CA,SecondCell_PSC,SecondCell_ARFCN,SecondCell_BAND,Site_BEARING,BATTERY,CHARGING,NRSTATE,R4,NTech1,NCellName1,NCellid1,NLAC1,NCell1,NARFCN1,NRxLev1,NQual1,NDistance1,NBearing1,NTech2,NCellName2,NCellid2,NLAC2,NCell2,NARFCN2,NRxLev2,NQual2,NDistance2,NBearing2,NTech3,NCellName3,NCellid3,NLAC3,NCell3,NARFCN3,NRxLev3,NQual3,NDistance3,NBearing3,NTech4,NCellName4,NCellid4,NLAC4,NCell4,NARFCN4,NRxLev4,NQual4,NDistance4,NBearing4,NTech5,NCellName5,NCellid5,NLAC5,NCell5,NARFCN5,NRxLev5,NQual5,NDistance5,NBearing5,NTech6,NCellName6,NCellid6,NLAC6,NCell6,NARFCN6,NRxLev6,NQual6,NDistance6,NBearing6,NTech7,NCellName7,NCellid7,NLAC7,NCell7,NARFCN7,NRxLev7,NQual7,NDistance7,NBearing7,NTech8,NCellName8,NCellid8,NLAC8,NCell8,NARFCN8,NRxLev8,NQual8,NDistance8,NBearing8,NTech9,NCellName9,NCellid9,NLAC9,NCell9,NARFCN9,NRxLev9,NQual9,NDistance9,NBearing9,NTech10,NCellName10,NCellid10,NLAC10,NCell10,NARFCN10,NRxLev10,NQual10,NDistance10,NBearing10,NTech11,NCellName11,NCellid11,NLAC11,NCell11,NARFCN11,NRxLev11,NQual11,NDistance11,NBearing11,NTech12,NCellName12,NCellid12,NLAC12,NCell12,NARFCN12,NRxLev12,NQual12,NDistance12,NBearing12,NTech13,NCellName13,NCellid13,NLAC13,NCell13,NARFCN13,NRxLev13,NQual13,NDistance13,NBearing13,NTech14,NCellName14,NCellid14,NLAC14,NCell14,NARFCN14,NRxLev14,NQual14,NDistance14,NBearing14,NTech15,NCellName15,NCellid15,NLAC15,NCell15,NARFCN15,NRxLev15,NQual15,NDistance15,NBearing15,NTech16,NCellName16,NCellid16,NLAC16,NCell16,NARFCN16,NRxLev16,NQual16,NDistance16,NBearing16,NTech17,NCellName17,NCellid17,NLAC17,NCell17,NARFCN17,NRxLev17,NQual17,NDistance17,NBearing17,NTech18,NCellName18,NCellid18,NLAC18,NCell18,NARFCN18,NRxLev18,NQual18,NDistance18,NBearing18,Unnamed: 259,Timestamp_seconds
0,2023-01-21 14:44:46,NaN,NaN,NaN,CLARO_BR,72405,724056481100018,NaN,113726,18,64811,4G,LTE,-110.0,-8.0,16.0,14.0,-61.0,1400.0,25.0,29.0,111.0,NaN,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,13,1,NaN,NaN,0,0,10.130.154.201,31.0,LOG START,NaN,NaN,NaN,8,12,1BC3E,FD2B,29113874,27.5,535,samsung:SM-G991B:TP1A.220624.014,L3,10/20/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,False,NaN,NaN,NaN,0,81,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1674312286
1,2023-01-21 14:44:46,NaN,NaN,NaN,CLARO_BR,72405,724056481100018,NaN,113726,18,64811,4G,LTE,-112.0,-8.0,17.0,14.0,-65.0,1400.0,119.0,53.0,111.0,NaN,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,13,1,NaN,NaN,0,0,10.130.154.201,31.0,PERIODIC,NaN,NaN,NaN,9,12,1BC3E,FD2B,29113874,27.5,535,samsung:SM-G991B:TP1A.220624.014,L3,10/20/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,False,NaN,NaN,NaN,0,81,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### YouTube QoE

In [7]:
YOUTUBE_QOE_PCAP_PATH = f'{DATA_PATH_PREFIX}/youtube-qoe-pcap'

def ft_comma_separated(value_type):
    def inner(value):
        value = [ e for e in value.split(',') ]
        value = [ value_type(v) for v in value ]
        return value
    return inner

def ft_comma_separated_ts(value_type):
    def inner(value):
        value = [ e.split(':') for e in value.split(',') ]
        value = [ { 'at': float(at), **value_type(*v) } for at, *v in value ]
        return value
    return inner

def youtube_qoe_field_transform(field, value):
    # https://developers.google.com/youtube/iframe_api_reference#getPlayerState
    player_state_enum_map = {
        'N':  'unstarted/cued', # -1 = unstarted; 5 = video cued
        'EN': 'ended',   # 0
        'PL': 'playing', # 1
        'PA': 'paused',  # 2
        'B':  'buffering', # 3
        'S':  'seek',    # guess based on player interactions
    }
    fields_transformers = {
        #'seq':   ft_identity(lambda  v: { 'value': int(v) }),
        #'event': ft_identity(lambda  v: { 'value': str(v) }),
        'cmt':   ft_comma_separated_ts(lambda  v: { 'value': float(v) }),
        'vps':   ft_comma_separated_ts(lambda  v: { 'value': player_state_enum_map.get(v, v)   }),
        'bh':    ft_comma_separated_ts(lambda  v: { 'value': float(v) }),
        'bwe':   ft_comma_separated_ts(lambda  v: { 'value': int(v)   }),
        'bwm':   ft_comma_separated_ts(lambda *v: { 'downloaded_bytes': int(v[0]), 'seconds_to_download': float(v[1]) }),
        'bat':   ft_comma_separated_ts(lambda *v: { 'percentage': float(v[0]) * 100, 'is_charging': v[1] == '1' }),
        'view':  ft_comma_separated_ts(lambda *v: { 'width': int(v[0]), 'height': int(v[1]) }),
        'df':    ft_comma_separated_ts(lambda  v: { 'value': int(v)   }),
    }

    if field not in fields_transformers:
        return value

    return fields_transformers[field](value)

def open_youtube_qoe_pcap(pcap_filename):
    # open_youtube_qoe_pcap
    res = pyshark.FileCapture(input_file=pcap_filename, display_filter='http2.header.value contains "api/stats/qoe"')

    # extract_youtube_qoe_urls_from_pcap
    res = [ { 'time': each.sniff_time, 'url': each.http2.headers_path } for each in res]

    # extract_youtube_qoe_metrics_from_urls
    res = [ { k: v for k, v in parse_qsl(urlparse(i['url']).query) + [('time', i['time'])] } for i in res ] # extract query params from URLs
    res = [ { k: youtube_qoe_field_transform(k, v) for k, v in i.items() } for i in res ] # transform each query param

    return res

datasets = [ file for file in os.listdir(YOUTUBE_QOE_PCAP_PATH) if file.endswith('.pcapng') ]

ytDf = pd.concat([ pd.DataFrame(open_youtube_qoe_pcap(f'{YOUTUBE_QOE_PCAP_PATH}/{dataset}')) for dataset in datasets ]).copy()
ytDf.rename(columns={ 'time': 'packet_time' }, inplace=True)
ytDf

,fmt,afmt,cpn,el,ns,fexp,cl,seq,docid,ei,event,plid,cbrand,cbr,cbrver,c,cver,cplayer,cmodel,cos,cosver,cplatform,vps,afs,vfs,view,bwe,vis,cmt,conn,bh,packet_time,user_intent,bwm,bat,df,ctmp
0,243,251,oMaRrct9XOA6WR0D,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,1,d5r-kEmQfBg,gA7vY83zJeyPobIPq_OcmAg,streamingstats,AAX03ncKwXUwJiIT,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 0.0, 'value': 'unstarted/cued'}, {'at'...",0.008:251::i,0.008:243:243::d,"[{'at': 0.008, 'width': 360, 'height': 203}]","[{'at': 0.008, 'value': 752419}]",0.008:0,"[{'at': 0.008, 'value': 0.0}]",0.008:1,"[{'at': 0.008, 'value': 0.0}]",2023-02-17 02:20:03.329911,NaN,NaN,NaN,NaN,NaN
1,243,251,oMaRrct9XOA6WR0D,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,2,d5r-kEmQfBg,gA7vY83zJeyPobIPq_OcmAg,streamingstats,AAX03ncKwXUwJiIT,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 0.037, 'value': 'buffering'}, {'at': 0...",NaN,NaN,NaN,"[{'at': 1.965, 'value': 909374}]",NaN,"[{'at': 0.037, 'value': 0.0}, {'at': 0.858, 'v...",NaN,"[{'at': 0.858, 'value': 7.401}, {'at': 1.965, ...",2023-02-17 02:20:05.273837,0,"[{'at': 1.965, 'downloaded_bytes': 1289567, 's...","[{'at': 1.965, 'percentage': 47.0, 'is_chargin...","[{'at': 1.965, 'value': 0}]",NaN
2,243,251,0p1K2OOv-yODQEOd,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,1,d5r-kEmQfBg,gw7vY-jvFq-i4dUPiO6EyAc,streamingstats,AAX03nc0skm3aacq,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 0.0, 'value': 'unstarted/cued'}, {'at'...",0.009:251::i,0.009:243:243::d,"[{'at': 0.009, 'width': 360, 'height': 203}]","[{'at': 0.009, 'value': 752419}]",0.009:0,"[{'at': 0.009, 'value': 0.0}]",0.009:1,"[{'at': 0.009, 'value': 0.0}]",2023-02-17 02:20:06.003438,NaN,NaN,NaN,NaN,NaN
3,243,251,0p1K2OOv-yODQEOd,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,2,d5r-kEmQfBg,gw7vY-jvFq-i4dUPiO6EyAc,streamingstats,AAX03nc0skm3aacq,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 0.023, 'value': 'buffering'}, {'at': 0...",NaN,NaN,NaN,"[{'at': 10.005, 'value': 1684310}]",NaN,"[{'at': 0.023, 'value': 0.0}, {'at': 0.293, 'v...",NaN,"[{'at': 0.293, 'value': 4.5}, {'at': 10.005, '...",2023-02-17 02:20:16.003221,0,"[{'at': 10.005, 'downloaded_bytes': 2661621, '...","[{'at': 10.005, 'percentage': 47.0, 'is_chargi...","[{'at': 10.005, 'value': 1}]",NaN
4,243,251,0p1K2OOv-yODQEOd,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,3,d5r-kEmQfBg,gw7vY-jvFq-i4dUPiO6EyAc,streamingstats,AAX03nc0skm3aacq,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 50.005, 'value': 'playing'}]",NaN,NaN,NaN,"[{'at': 50.005, 'value': 1277198}]",NaN,"[{'at': 50.005, 'value': 49.58}]",NaN,"[{'at': 50.005, 'value': 82.341}]",2023-02-17 02:20:56.005722,NaN,"[{'at': 50.005, 'downloaded_bytes': 5020251, '...","[{'at': 50.005, 'percentage': 47.0, 'is_chargi...","[{'at': 50.005, 'value': 1}]",NaN
5,243,251,0p1K2OOv-yODQEOd,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,4,d5r-kEmQfBg,gw7vY-jvFq-i4dUPiO6EyAc,streamingstats,AAX03nc0skm3aacq,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,sm-g991b,Android,13,MOBILE,"[{'at': 80.005, 'value': 'playing'}]",NaN,NaN,NaN,"[{'at': 80.005, 'value': 720374}]",NaN,"[{'at': 80.005, 'value': 79.58}]",NaN,"[{'at': 80.005, 'value': 120.593}]",2023-02-17 02:21:26.004351,NaN,"[{'at': 80.005, 'downloaded_bytes': 4284958, '...","[{'at': 80.005, 'percentage': 47.0, 'is_chargi...",NaN,NaN
6,243,251,0p1K2OOv-yODQEOd,detailpage,yt,"23848211,23983296,24004644,24007246,24080738,2...",509820939,5,d5r-kEmQfBg,gw7vY-jvFq-i4dUPiO6EyAc,streamingstats,AAX03nc0skm3aacq,samsung,Chrome Mobile,109.0.0.0,MWEB,2.20230215.10.00,UNIPLAYER,s